# Task 1.1
Discretización de la imagen

In [ ]:
! pip install opencv-contrib-python

In [16]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('nput.bmp')

def convert_image_to_cells(image, cell_size):
    """Converts an image to cells based on color.

    Args:
        image: A NumPy array representing the image.
        cell_size: The size of each cell in pixels.

    Returns:
        A NumPy array representing the cells, where each element is the average color
        of the corresponding cell in the original image.
    """

    # Check if the image is a 3-channel BGR image
    if len(image.shape) != 3 or image.shape[2] != 3:
        raise ValueError("Image must be a 3-channel BGR image.")

    height, width = image.shape[:2]  # Unpack the width and height
    cells = np.zeros((height // cell_size, width // cell_size, 3), dtype=np.uint8)

    for i in range(height // cell_size):
        for j in range(width // cell_size):
            # Get the slice of the image corresponding to the current cell
            cell_image = image[i * cell_size : (i + 1) * cell_size,
                               j * cell_size : (j + 1) * cell_size]

            # Calculate the average color of the cell
            cell_color = np.mean(cell_image, axis=(0, 1))

            # Assign the average color to the corresponding cell in the output array
            cells[i, j] = cell_color

    return cells

# Convert the image to cells
cells = convert_image_to_cells(image, cell_size=20)

# Save the cells to a file (optional)
cv2.imwrite("cells.bmp", cells)

print(cells.shape)


(29, 29, 3)
